In [1]:
%load_ext zipline
%matplotlib inline

import zipline as zp
import pandas as pd
from datetime import datetime
import sys
import os
from zipline import run_algorithm
from zipline.api import order_target_percent, order_target, symbol, sid
from zipline.utils.calendars import get_calendar
from datetime import datetime
import pytz
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', None)
pd.set_option('mode.chained_assignment', 'raise')
pd.set_option('display.max_columns', None)

sys.path.append(os.path.dirname(os.getcwd())) # so we can import stockalyzer

""" TODO:
    - Dividend & Split data (fundamentals)
    - SID for stocks (fundamentals)
    - Timezones!"""

' TODO:\n    - Dividend & Split data (fundamentals)\n    - SID for stocks (fundamentals)\n    - Timezones!'

In [17]:
from zipline.data.bundles import load, register
from zipline.data.data_portal import DataPortal
from zipline.utils.calendars import get_calendar
import zipline.data.bundles.sp500_stock_data as x

In [18]:
x.sp500_data

<module 'zipline.data.bundles.sp500_stock_data' from '/Users/jfeibs/.pyenv/versions/3.5.10/envs/venv3.5/lib/python3.5/site-packages/zipline/data/bundles/sp500_stock_data.py'>

In [32]:
symbols = ['AMZN','TSLA']
register('sp500_data', symbols)
sp500_bundle = load('sp500_data')
start_dt = pd.Timestamp('2019-06-04', tz = 'utc')
end_dt = pd.Timestamp('2021-03-15', tz = 'utc')
sp500_data = DataPortal(sp500_bundle.asset_finder, #get_calendar('NYSE'),
                    trading_calendar=trading_calendar,
                       sp500_bundle.equity_daily_bar_reader.first_trading_day,
                       equity_minute_reader=sp500_bundle.equity_minute_bar_reader,
                       equity_daily_reader=sp500_bundle.equity_daily_bar_reader,
                       adjustment_reader=sp500_bundle.adjustment_reader)

sp_symbols = []
for ticker in symbols:
    sp_symbols.append(sp500_data.asset_finder.lookup_symbol(ticker,end_dt))
pricing = sp500_data.get_history_window(sp_symbols,end_dt,1000,'1d','close')
pricing.asfreq('D').dropna().plot()

SyntaxError: positional argument follows keyword argument (<ipython-input-32-8224e8efe33d>, line 8)

In [25]:
start_dt

Timestamp('2019-06-04 00:00:00+0000', tz='UTC')

In [27]:
import stockalyzer as st
from stockalyzer.data.data_model import TDDataFetcher
from stockalyzer.data.data_manager import StockDataManager
# from stockalyzer.strategies.ema_crossover import EMACrossoverStrategy

In [17]:
tickers = ["AMZN", "TSLA", "GOOG"]
# sm = StockDataManager(TDDataFetcher(tickers), datetime(2019, 6, 1))
# output_files = sm.export_csv()
# cerebro, strats = sm.run_strategy(EMACrossoverStrategy, [20,50], plot=True)


In [31]:
def initialize(context):
    context.stocks = [symbol(t) for t in tickers]
    context.m = len(tickers)
    context.i = 0
    context.stock = symbol("AMZN")
    
def handle_data(context, data):
    # Skip first 100 days to get full windows
#     context.i += 1
#     if context.i < 100:
#         return
    order_target(symbol("TSLA"),100)
    # Compute averages
    # data.history() has to be called with the same params
    # from above and returns a pandas dataframe.
    
#     for stock in context.stocks:
#         short_mavg = data.history(stock, 'price', bar_count=50, frequency="1d").mean()
#         long_mavg = data.history(stock, 'price', bar_count=100, frequency="1d").mean()
#         # Trading logic
#         if short_mavg > long_mavg:
#             # order_target orders as many shares as needed to
#             # achieve the desired number of shares.
#             order_target(stock, 10)
#         elif short_mavg < long_mavg:
#             order_target(stock, 0)
    
def analyze(context):
    fig = plt.figure(figsize=(12, 8))
    
    # First chart
    ax = fig.add_subplot(311)
    ax.set_title('Strategy Results')
    ax.plot(perf['portfolio_value'], linestyle='-', 
                label='Equity Curve', linewidth=1.0)
    ax.legend()
    ax.grid(False)
    
    # Second chart
    ax = fig.add_subplot(312)
    ax.plot(perf['gross_leverage'], 
            label='Exposure', linestyle='-', linewidth=1.0)
    ax.legend()
    ax.grid(True)
    # Third chart
    ax = fig.add_subplot(313)
    ax.plot(perf['returns'], label='Returns', linestyle='-.', linewidth=1.0)
    ax.legend()
    ax.grid(True)
    plt.savefig('strategy',dpi=400)
    
    
zp.data.bundles.load('sp500_data')

# Fire off the backtest
results = run_algorithm(
    start=pd.Timestamp(sm.start_date, tz=pytz.utc), 
    end=pd.Timestamp(sm.end_date, tz=pytz.utc), 
    initialize=initialize, 
    analyze=analyze,
    handle_data=handle_data, 
    capital_base=10000, 
    trading_calendar=get_calendar('NYSE'),
    data_frequency = 'daily', 
    bundle='sp500_data' 
)

BundleData(asset_finder=<zipline.assets.assets.AssetFinder object at 0x122ead908>, equity_minute_bar_reader=<zipline.data.minute_bars.BcolzMinuteBarReader object at 0x125238550>, equity_daily_bar_reader=<zipline.data.bcolz_daily_bars.BcolzDailyBarReader object at 0x12523f0b8>, adjustment_reader=<zipline.data.adjustments.SQLiteAdjustmentReader object at 0x12524aeb8>)

BundleData(asset_finder=<zipline.assets.assets.AssetFinder object at 0x122ead908>, equity_minute_bar_reader=<zipline.data.minute_bars.BcolzMinuteBarReader object at 0x125238550>, equity_daily_bar_reader=<zipline.data.bcolz_daily_bars.BcolzDailyBarReader object at 0x12523f0b8>, adjustment_reader=<zipline.data.adjustments.SQLiteAdjustmentReader object at 0x12524aeb8>)

BundleData(asset_finder=<zipline.assets.assets.AssetFinder object at 0x122ead908>, equity_minute_bar_reader=<zipline.data.minute_bars.BcolzMinuteBarReader object at 0x125238550>, equity_daily_bar_reader=<zipline.data.bcolz_daily_bars.BcolzDailyBarReader object at 0x12523f0b8>, adjustment_reader=<zipline.data.adjustments.SQLiteAdjustmentReader object at 0x12524aeb8>)

KeyError: Equity(1 [TSLA])

KeyError: Equity(1 [TSLA])

KeyError: Equity(1 [TSLA])

In [20]:
# path = os.path.dirname(os.getcwd()) + "/data/SP500"
# symbols = [f[:-4].split('-') for f in os.listdir(path) if f[0] != '.']

# for symbol, sid in (symbols):
# #     print("{}:{}".format(sid,symbol))
#     print(symbol)
#     print(sid)

# # print("OK")
# # symbols

In [14]:
perf = zp.run_algorithm(start=pd.Timestamp(sm.start_date, tz=pytz.utc), end=pd.Timestamp(sm.end_date, tz=pytz.utc), initialize=initialize, capital_base=100000, handle_data=handle_data, bundle='sp500_data')

ValueError: At least one valid asset id is required.

ValueError: At least one valid asset id is required.

ValueError: At least one valid asset id is required.

In [ ]:


# BACKTRADER

# plts = []
# for ticker in tickers:
#     plt = sm.run_strategy_for(ticker, EMACrossoverStrategy, [20,50])[0][0]
#     plt./



# for x in strats[0].analyzers:
#     print(x)
#     print(x.get_analysis())
#     print('\n')

## TODO
- Structure to track performance of individual securities (multi data feed backtrader)
- 
- 
- Implement Machine Learning (regression, LSTM)

In [23]:
print("hi")

hi
hi
hi
